# 2. Modélisation avec Vertex AI Forecast - Chicago Taxi Demand

Ce notebook couvre l'entraînement d'un modèle de prévision de la demande de taxis à Chicago en utilisant Vertex AI Forecast. Nous allons :
- Initialiser l'environnement Vertex AI
- Charger la configuration depuis le fichier YAML
- Créer un dataset de séries temporelles à partir des données BigQuery
- Configurer et lancer un job d'entraînement AutoML Forecasting
- Analyser les résultats et les métriques d'évaluation

## 1. Configuration et Initialisation

Importons les bibliothèques nécessaires et initialisons l'environnement.

In [1]:
# Bibliothèques standards
import os
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Google Cloud & Vertex AI
from google.cloud import aiplatform
from google.cloud import bigquery

# Configuration visuelle
%matplotlib inline
sns.set(style="whitegrid", context="talk")
plt.rcParams['figure.figsize'] = [12, 8]

## 2. Configuration du Projet GCP

Définissons les identifiants du projet et initialisons Vertex AI.

In [11]:
# Configuration du projet GCP
PROJECT_ID = "avisia-certification-ml-yde"  # Remplacez par votre Project ID
REGION = "europe-west1"  # Région pour Vertex AI (assurez-vous qu'elle supporte Forecast)
BUCKET_URI = f"gs://{PROJECT_ID}-vertex-bucket"  # URI du bucket GCS
BQ_DATASET = "chicago_taxis"
BQ_SOURCE_URI = f"bq://{PROJECT_ID}.{BQ_DATASET}.demand_by_hour"

# Initialisation de Vertex AI
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

print(f"✅ Vertex AI initialisé avec succès pour le projet {PROJECT_ID}")

✅ Vertex AI initialisé avec succès pour le projet avisia-certification-ml-yde


## 3. Chargement de la Configuration

Chargeons les paramètres de configuration depuis le fichier YAML.

In [3]:
# Chargement du fichier de configuration
try:
    with open("../config/pipeline_config.yaml", "r") as f:
        config = yaml.safe_load(f)
    print("✅ Configuration chargée avec succès.")
except FileNotFoundError:
    try:
        with open("config/pipeline_config.yaml", "r") as f:
            config = yaml.safe_load(f)
        print("✅ Configuration chargée avec succès.")
    except FileNotFoundError:
        print("⚠️ Fichier de configuration introuvable. Utilisation des valeurs par défaut.")
        config = {
            "forecasting": {
                "time_column": "timestamp_hour",
                "target_column": "trip_count",
                "context_column": "pickup_community_area",
                "forecast_horizon": 24,
                "window_size": 168,
                "available_at_forecast": [
                    "timestamp_hour", "day_of_year", "day_of_week", "hour", 
                    "month", "is_weekend"
                ],
                "unavailable_at_forecast": ["trip_count"],
                "data_granularity_unit": "hour"
            },
            "vertex_ai_forecast": {
                "display_name": "chicago_taxi_forecast_model",
                "optimization_objective": "minimize-rmse",
                "budget_milli_node_hours": 100
            }
        }

# Extraction des paramètres de configuration
forecast_config = config["forecasting"]
vertex_config = config["vertex_ai_forecast"]

# Paramètres clés pour le forecasting
time_column = forecast_config["time_column"]
target_column = forecast_config["target_column"]
context_column = forecast_config["context_column"]
forecast_horizon = forecast_config["forecast_horizon"]
window_size = forecast_config["window_size"]
available_at_forecast = forecast_config["available_at_forecast"]
unavailable_at_forecast = forecast_config["unavailable_at_forecast"]

# Affichage des principaux paramètres
print(f"\nParamètres de forecasting:")
print(f"- Colonne temporelle: {time_column}")
print(f"- Colonne cible: {target_column}")
print(f"- Identifiant de série: {context_column}")
print(f"- Horizon de prévision: {forecast_horizon} heures")
print(f"- Taille de la fenêtre historique: {window_size} heures")

✅ Configuration chargée avec succès.

Paramètres de forecasting:
- Colonne temporelle: timestamp_hour
- Colonne cible: trip_count
- Identifiant de série: pickup_community_area
- Horizon de prévision: 24 heures
- Taille de la fenêtre historique: 168 heures


## 4. Création du Dataset de Séries Temporelles

Créons un dataset de séries temporelles dans Vertex AI à partir de la table BigQuery.

In [12]:
# Nom d'affichage du dataset
dataset_display_name = f"{vertex_config['display_name']}-dataset"

# Création du dataset
try:
    dataset = aiplatform.TimeSeriesDataset.create(
        display_name=dataset_display_name,
        bq_source=BQ_SOURCE_URI,
    )
    print(f"✅ Dataset créé: {dataset.resource_name}")
except Exception as e:
    print(f"⚠️ Erreur lors de la création du dataset: {e}")
    print("Tentative de récupération d'un dataset existant...")
    try:
        # Rechercher parmi les datasets existants
        datasets = aiplatform.TimeSeriesDataset.list(
            filter=f"display_name={dataset_display_name}",
            order_by="create_time desc"
        )
        if datasets:
            dataset = datasets[0]
            print(f"✅ Dataset existant récupéré: {dataset.resource_name}")
        else:
            print("❌ Aucun dataset existant trouvé.")
    except Exception as e2:
        print(f"❌ Erreur lors de la récupération du dataset: {e2}")

Creating TimeSeriesDataset
Create TimeSeriesDataset backing LRO: projects/807699310940/locations/europe-west1/datasets/7446095053438582784/operations/679736505110888448
Create TimeSeriesDataset backing LRO: projects/807699310940/locations/europe-west1/datasets/7446095053438582784/operations/679736505110888448
TimeSeriesDataset created. Resource name: projects/807699310940/locations/europe-west1/datasets/7446095053438582784
To use this TimeSeriesDataset in another session:
ds = aiplatform.TimeSeriesDataset('projects/807699310940/locations/europe-west1/datasets/7446095053438582784')
✅ Dataset créé: projects/807699310940/locations/europe-west1/datasets/7446095053438582784


## 5. Configuration et Lancement du Job d'Entraînement

Configurons et lançons un job d'entraînement AutoML Forecasting.

In [ ]:
# Nom du job d'entraînement
job_display_name = f"taxi_demand_forecast_job_{datetime.now().strftime('%Y%m%d_%H%M')}"

# Configuration des transformations de colonnes
column_transformations = vertex_config.get("column_transformations", {})

# Définition MANUELLE des transformations pour éviter les problèmes avec pickup_community_area
formatted_transformations = []

# Si les transformations sont définies dans la configuration, les utiliser
if column_transformations:
    # Conversion du format dict en format attendu par l'API
    for col, transform_type in column_transformations.items():
        # Ne pas ajouter de transformation pour context_column (pickup_community_area)
        if col != context_column:
            formatted_transformations.append({transform_type: {"column_name": col}})
else:
    # Transformations automatiques pour les colonnes autres que context_column
    formatted_transformations.append({"auto": {"column_name": target_column}})
    formatted_transformations.append({"auto": {"column_name": time_column}})
    
    # Ajouter les colonnes disponibles à l'heure de la prévision (sauf context_column)
    for col in available_at_forecast:
        if col not in [target_column, time_column, context_column]:
            formatted_transformations.append({"auto": {"column_name": col}})

print(f"Transformations de colonnes configurées: {len(formatted_transformations)} colonnes")
print(f"Note: {context_column} est EXCLU des transformations pour éviter les conflits")

Transformations de colonnes configurées: 11 colonnes


In [14]:
# Création du job d'entraînement
training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=job_display_name,
    optimization_objective=vertex_config.get("optimization_objective", "minimize-rmse"),
    column_transformations=formatted_transformations,
)

print(f"✅ Job d'entraînement configuré: {job_display_name}")

✅ Job d'entraînement configuré: taxi_demand_forecast_job_20250409_1736


In [ ]:
# Nom du modèle
model_display_name = f"{vertex_config['display_name']}_{datetime.now().strftime('%Y%m%d_%H%M')}"

# S'assurer que context_column n'est PAS dans available_at_forecast ni unavailable_at_forecast
if context_column in available_at_forecast:
    available_at_forecast.remove(context_column)
    print(f"Retrait de {context_column} des colonnes disponibles pour la prévision")

if context_column in unavailable_at_forecast:
    unavailable_at_forecast.remove(context_column)
    print(f"Retrait de {context_column} des colonnes indisponibles pour la prévision")

# Lancement de l'entraînement
print(f"⏳ Démarrage de l'entraînement du modèle... Cela peut prendre plusieurs heures.")
try:
    model = training_job.run(
        dataset=dataset,
        target_column=target_column,
        time_column=time_column,
        time_series_identifier_column=context_column,
        unavailable_at_forecast_columns=unavailable_at_forecast,
        available_at_forecast_columns=available_at_forecast,
        forecast_horizon=forecast_horizon,
        context_window=window_size,
        data_granularity_unit=forecast_config.get("data_granularity_unit", "hour"),
        data_granularity_count=1,
        budget_milli_node_hours=vertex_config.get("budget_milli_node_hours", 100),
        model_display_name=model_display_name,
        training_fraction_split=vertex_config.get("training_fraction_split", 0.8),
        validation_fraction_split=vertex_config.get("validation_fraction_split", 0.1),
        test_fraction_split=vertex_config.get("test_fraction_split", 0.1),
        export_evaluated_data_items=True,
        sync=True,  # Mode synchrone: attend la fin de l'entraînement
    )
    print(f"✅ Modèle entraîné avec succès: {model.display_name}")
    print(f"Resource name: {model.resource_name}")
except Exception as e:
    print(f"❌ Erreur lors de l'entraînement du modèle: {e}")
    print("\nDétails complets de l'erreur pour analyse:")
    import traceback
    print(traceback.format_exc())

Retrait de pickup_community_area des colonnes disponibles pour la prévision
⏳ Démarrage de l'entraînement du modèle... Cela peut prendre plusieurs heures.
❌ Erreur lors de l'entraînement du modèle: 400 The columns 'pickup_community_area' have transformation but do not have any time dependency properties (time series attribute, unavailable at forecast, available at forecast).


## 6. Évaluation du Modèle

Examinons les métriques d'évaluation du modèle entraîné.

In [8]:
try:
    # Récupération des métriques d'évaluation
    evaluation = model.get_model_evaluation()
    metrics = evaluation.metrics
    
    print("Métriques d'évaluation :")
    for metric_name, metric_value in metrics.items():
        if isinstance(metric_value, (int, float)):
            print(f"- {metric_name}: {metric_value:.4f}")
        else:
            print(f"- {metric_name}: {metric_value}")
except Exception as e:
    print(f"⚠️ Impossible de récupérer les métriques d'évaluation: {e}")
    print("Les métriques peuvent ne pas être disponibles immédiatement après l'entraînement.")

⚠️ Impossible de récupérer les métriques d'évaluation: name 'model' is not defined
Les métriques peuvent ne pas être disponibles immédiatement après l'entraînement.


## 7. Importance des Features

Analysons l'importance des features dans le modèle.

In [9]:
try:
    # Récupération de l'importance des features
    feature_importance = model.get_feature_importance()
    
    if feature_importance:
        # Conversion en DataFrame pour faciliter la visualisation
        feature_importance_df = pd.DataFrame({
            'Feature': [item.feature_id for item in feature_importance],
            'Importance': [item.importance_score for item in feature_importance]
        })
        
        # Tri par importance décroissante
        feature_importance_df = feature_importance_df.sort_values('Importance', ascending=False)
        
        # Affichage
        print("Importance des features :")
        print(feature_importance_df)
        
        # Visualisation
        plt.figure(figsize=(12, 8))
        sns.barplot(x='Importance', y='Feature', data=feature_importance_df)
        plt.title('Importance des Features', fontsize=16)
        plt.xlabel('Importance', fontsize=14)
        plt.ylabel('Feature', fontsize=14)
        plt.tight_layout()
        plt.show()
    else:
        print("⚠️ Aucune information d'importance des features disponible.")
except Exception as e:
    print(f"⚠️ Impossible de récupérer l'importance des features: {e}")

⚠️ Impossible de récupérer l'importance des features: name 'model' is not defined


## 8. Sauvegarde des Informations du Modèle

Sauvegardons les informations du modèle pour une utilisation ultérieure.

In [10]:
# Sauvegarde des informations du modèle dans un fichier pour référence ultérieure
try:
    model_info = {
        'model_name': model.display_name,
        'resource_name': model.resource_name,
        'create_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'project': PROJECT_ID,
        'region': REGION,
        'forecast_horizon': forecast_horizon,
        'window_size': window_size,
    }
    
    # Création du répertoire si nécessaire
    os.makedirs('outputs', exist_ok=True)
    
    # Sauvegarde dans un fichier YAML
    with open(f'outputs/model_info_{datetime.now().strftime("%Y%m%d_%H%M")}.yaml', 'w') as f:
        yaml.dump(model_info, f)
    
    print(f"✅ Informations du modèle sauvegardées.")
except Exception as e:
    print(f"⚠️ Impossible de sauvegarder les informations du modèle: {e}")

⚠️ Impossible de sauvegarder les informations du modèle: name 'model' is not defined


## 9. Conclusion et Prochaines Étapes

### Résumé
- Nous avons créé un dataset de séries temporelles à partir des données BigQuery
- Nous avons configuré et entraîné un modèle de forecasting avec Vertex AI AutoML
- Nous avons évalué les performances du modèle et analysé l'importance des features

### Prochaines Étapes
- Utiliser le modèle pour générer des prédictions batch sur des périodes futures
- Visualiser et analyser les résultats de prévision
- Explorer d'autres configurations de modèle pour améliorer les performances

Le notebook suivant (3_Batch_Prediction_Visualization.ipynb) abordera la génération et la visualisation des prédictions.